In [1]:
from tqdm import tqdm
from num2words import num2words
from collections import Counter
from scipy import spatial, sparse
from nltk.corpus import stopwords
from itertools import combinations
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from num2words import num2words
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import spacy
import imp, gzip
import nltk
import pickle
import scipy
import gensim
import multiprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import utils as my_utils

In [3]:
stop_words = stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()
stemmer = nltk.stem.PorterStemmer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

In [4]:
def get_edges(i):
    t = np.where(i>0)[0]
    comb = combinations(t, 2)    
    embeds = {j:[] for j in t}

    for p, q in comb:
        if word_similarity[p][q]:
            embeds[p] += [q]
            embeds[q] += [p]
    return embeds

In [5]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in tqdm(parse(path)):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [6]:
def process_df(df):
    df['text'] = myutils.preprocess(df['reviewText'])
    return df

In [7]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r', encoding='utf8')
    model = {}
    for line in tqdm(f):
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

In [8]:
dataset_ = getDF('datasets/reviews_Electronics_5.json.gz')
dataset_.shape

1689188it [01:17, 21728.02it/s]


(1689188, 9)

In [9]:
embedding_dim = 300
embeddings_index = loadGloveModel("nongit_resources/glove.6B.300d.txt")

2414it [00:00, 12142.95it/s]

Loading Glove Model


400000it [00:36, 10848.52it/s]

Done. 400000  words loaded!


In [10]:
cutoff = .35
title = "electronics_glove"

min_df = 5
max_df = .5
max_features = 50000

n_cores = 30
n_docs = 10000

In [11]:
dataset = dataset_.head(n_docs)

In [12]:
n = int(dataset.shape[0]/n_cores)
list_df = [dataset[i:i+n] for i in range(0, dataset.shape[0],n)]

pool = multiprocessing.Pool(n_cores)
processed_list_df = pool.map(process_df, list_df)
pool.close()

dataset = pd.concat(processed_list_df)
dataset.shape

NameError: name 'myutils' is not defined

In [ ]:
vectorizer = CountVectorizer(analyzer="word",tokenizer=None,preprocessor=None,
                             stop_words="english", max_features=max_features,
                             max_df=max_df, min_df=min_df)

In [ ]:
wordOccurenceMatrix = vectorizer.fit_transform(dataset.text.tolist()).toarray()
words = vectorizer.get_feature_names()

In [ ]:
word_embeddings = []

for word in tqdm(words):
    emb = embeddings_index.get(word, np.array([0]*embedding_dim))
    word_embeddings.append(emb.tolist())

word_embeddings = np.array(word_embeddings)

In [ ]:
word_similarity = cosine_similarity(word_embeddings)
word_similarity = word_similarity > cutoff
word_similarity = word_similarity.astype(int)
np.fill_diagonal(word_similarity, 0)

In [ ]:
wordOccuranceMatrixBinary = wordOccurenceMatrix.copy()
wordOccuranceMatrixBinary[wordOccuranceMatrixBinary > 1] = 1
wordOccuranceMatrixBinary.shape

In [ ]:
pp = np.array([i[1] for i in nltk.pos_tag(words)])
pp[pp=='JJ'] = 1
pp[pp=='JJR'] = 1
pp[pp=='JJS'] = 1
pp[pp=='NN'] = 1
pp[pp=='NNS'] = 1
pp[pp=='NNP'] = 1
pp[pp=='NNPS'] = 1
pp[pp!='1'] = 0
pp = pp.astype(int)

In [ ]:
wordOccuranceMatrixBinary[:, np.where(pp!=1)[0]] = 0

In [ ]:
%%time
pool = multiprocessing.Pool(n_cores)
similar_words = pool.map(get_edges, wordOccuranceMatrixBinary)
pool.close()
pickle_out = open("resources/"+ title +"_" + str(cutoff) + "_edges.pickle","wb")
pickle.dump(similar_words, pickle_out)
pickle_out.close()

In [ ]:
dataset.to_pickle("resources/"+ title + "_" + str(cutoff) +"_dataset")

### Appendix

In [ ]:
# wordOccuranceMatrixBinary[0].sum()

# np.sum(wordOccuranceMatrixBinary)

# Counter(np.array([i[1] for i in nltk.pos_tag(words)]))

# pp.sum()

# np.where(pp!=1)[0].shape

In [ ]:
#     pd = pd.apply(lambda x: convert_numbers(x))

In [ ]:
# def process_df(df):
#     df['text'] = preprocess(df['reviewText'])
    
# #     pool = multiprocessing.Pool(n_cores)
# #     df['cleaned'] = pool.map(process_l, df['text'].tolist())
# #     pool.close()
    
# #     df['text'] = df['cleaned'].apply(lambda x: " ".join(x))
#     return df

In [ ]:
# p = [item for sublist in dataset['cleaned'].tolist() for item in sublist]

In [ ]:
# sorted(Counter(p))

In [ ]:
# def process_l(s):
#     return [i.lemma_ for i in sp(s) if i.lemma_ not in '-PRON-']

In [ ]:
# l = dataset['text'].tolist()

In [ ]:
# pool = multiprocessing.Pool(n_cores)
# processed_l = pool.map(process_l, l)
# pool.close()

In [ ]:
# joblib.dump(sampler, "resources/sampler_20iter_0.5_1")

In [ ]:
# pickle_out = open("resources/amazon_muiscal_glove_0.4.pickle","wb")
# pickle.dump(similar_words, pickle_out)
# pickle_out.close()